In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples

import pandas as pd
import numpy as np
import polars as pl
import datetime
import seaborn as sns
import lightgbm as lgb
import random
import matplotlib.pyplot as plt
import logging
import time
import re

In [2]:
df = pl.read_parquet("./notebook-analysis_df_23jul25_1912.parquet")

In [4]:
out =  (
    df.sort("mean_pred")
      .with_columns(
          tail_mean_return = (
              pl.col("mean_return").cum_sum(reverse=True) /
              pl.arange(1, pl.len() + 1).reverse()
          )
      )
)

In [5]:
out2 =  (
    df.sort("end_train_date")
      .with_columns(
          tail_pred_return = (
              pl.col("mean_pred").cum_sum(reverse=True) /
              pl.arange(1, pl.len() + 1).reverse()
          )
      )
)

In [6]:
out3 =  (
    df.sort("end_train_date")
      .with_columns(
          tail_pred_return = (
              pl.col("mean_return").cum_sum(reverse=True) /
              pl.arange(1, pl.len() + 1).reverse()
          )
      )
)

In [7]:
out4 =  (
    df.sort("end_train_date")
      .with_columns(
          roll_mean_return = pl.col("mean_return").rolling_mean(window_size=40, min_periods=1, center=False)
          )# Adjust window size as needed
)

C:\Users\kimer\AppData\Local\Temp\ipykernel_17008\909638669.py:4: DeprecationWarning: The argument `min_periods` for `Expr.rolling_mean` is deprecated. It has been renamed to `min_samples`.
  roll_mean_return = pl.col("mean_return").rolling_mean(window_size=40, min_periods=1, center=False)


In [8]:
out5 = (
    df.filter(pl.col("mean_pred") > pl.quantile("mean_pred",0.2))
    .sort("end_train_date")
      .with_columns(
          roll_mean_return = pl.col("mean_return").rolling_mean(window_size=10, min_periods=1, center=False)
          )# Adjust window size as needed
    )

C:\Users\kimer\AppData\Local\Temp\ipykernel_17008\835554040.py:5: DeprecationWarning: The argument `min_periods` for `Expr.rolling_mean` is deprecated. It has been renamed to `min_samples`.
  roll_mean_return = pl.col("mean_return").rolling_mean(window_size=10, min_periods=1, center=False)
